In [1]:
###
### Pull data via the YouTube Data API v3 on a single channel
### Utilize the Requests library to pull JSON response from API
### Load desired results to a Pandas DataFrame for analysis, loading to SQL, etc
###

# import libraries
import pandas as pd
import requests
import time
import keyring

# Keys
API_KEY = keyring.get_password('YouTube_API','API_KEY')
CHANNEL_ID = 'UCjOuL9q-0UIh21wfLJ21h-Q' #channel ID for Sonio

In [2]:
#api call for specific video statistics
def get_video_details(video_id):
    url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=contentDetails,statistics&key='+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    video_duration = response_video_stats['items'][0]['contentDetails']['duration']
    video_duration = str(video_duration).split('PT')[1]
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount'] #dislike count was made private as of 2021-12-31
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return video_duration, view_count, like_count, comment_count

In [3]:
def get_videos(df):
    url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=10000'
    response = requests.get(url).json()
    time.sleep(3) #allow time for response

    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            channel_name = video['snippet']['channelTitle']
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace('&amp;','')
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split('T')[0]

            video_duration, view_count, like_count, comment_count = get_video_details(video_id)

            # append video data as records in pandas df
            video_df = pd.DataFrame.from_dict({
                'channel_name':[channel_name],
                'upload_date':[upload_date],
                'video_duration':[video_duration],
                'video_id':[video_id],
                'video_title':[video_title],
                'view_count':[view_count],
                'like_count':[like_count],
                'comment_count':[comment_count]
            })

            df = pd.concat([df,video_df], ignore_index=True)
    return df


In [ ]:
#initialize dataframe
df = pd.DataFrame(columns=['channel_name','upload_date','video_duration','video_id','video_title','view_count','like_count','comment_count'])

df = get_videos(df)